### Imports

In [1]:
from datetime import datetime
import time

from contracts_lib_py.account import Account
from common_utils_py.agreements.service_types import ServiceTypesIndices


from nevermined_sdk_py import Config, Nevermined
from nevermined_sdk_py.nevermined.keeper import NeverminedKeeper as Keeper

import utils

CONSUMER_ADDRESS = "0x00Bd138aBD70e2F00903268F3Db08f2D25677C9e"
CONSUMER_PASSWORD = "node0"
CONSUMER_KEYFILE = "../resources/accounts/consumer.json"
CONFIG_FILE = "../config.ini"

COORDINATOR_URL = "http://172.17.0.2:8081"

PROVIDER_ADDRESS = "0x068Ed00cF0441e4829D9784fCBe7b9e26D4BD8d0"
ASSET_COMPUTE_DID_1 = "did:nv:ef129ded18f8bd94096156981726bab4e2f8afe7510617d33beb14e9cbafe17f"
ASSET_COMPUTE_DID_2 = "did:nv:a7f831260c3fd8cc3b4cdac657d50a61860f380eb3829082f6b6719dcff770db"
ASSET_COORDINATOR_DID = "did:nv:cad3d86be04a00bf7bfd5ff0aa4009dd66f904a84f971c5d3a1b8f852fcabeed"

ModuleNotFoundError: No module named 'workshop'

### Setup Nevermined and accounts

In [2]:
nevermined = Nevermined(Config(CONFIG_FILE))
consumer_account = Account(CONSUMER_ADDRESS, CONSUMER_PASSWORD, CONSUMER_KEYFILE)

nevermined.accounts.request_tokens(consumer_account, 10)


True

### Publish algorithm

In [3]:
metadata_algorithm = utils.metadata_algorithm_template()

metadata_algorithm["main"]["dateCreated"] = utils.date_now()
metadata_algorithm["main"]["files"][0]["url"] = "https://github.com/adap/flower/raw/main/examples/quickstart_tensorflow/client.py"

metadata_algorithm["main"]["algorithm"]["entrypoint"] = (
    "pip install tensorflow-cpu flwr==0.15 && "
    "python client.py 172.17.0.2:8081"
)

metadata_algorithm["main"]["algorithm"]["requirements"]["container"]["image"] = "python"
metadata_algorithm["main"]["algorithm"]["requirements"]["container"]["tag"] = "3.8-slim-buster"

utils.print_json(metadata_algorithm)

ddo_algorithm = nevermined.assets.create(metadata_algorithm, consumer_account, providers=[PROVIDER_ADDRESS])
print(f"Algorithm DID: {ddo_algorithm.did}")

{
  "main": {
    "name": "Fraud Demo Algorithm",
    "dateCreated": "2021-03-11T12:48:22Z",
    "author": "Nevermined Consumer",
    "license": "",
    "price": "0",
    "files": [
      {
        "index": 0,
        "contentType": "text/text",
        "checksum": "0x52b5c93b82dd9e7ecc3d9fdf4755f7f69a54484941897dc517b4adfe3bbc3377",
        "checksumType": "MD5",
        "contentLength": "12057507",
        "url": "https://github.com/nevermined-io/hellodecentralization/raw/main/notebooks/federated_fraud_demo.ipynb"
      }
    ],
    "type": "algorithm",
    "algorithm": {
      "language": "python",
      "format": "py",
      "version": "0.1.0",
      "entrypoint": "pip install jupyter pandas papermill scikit-learn xain-sdk && papermill --stdout-file - federated_fraud_demo.ipynb $NEVERMINED_OUTPUTS_PATH/federated_fraud_demo_output.ipynb",
      "requirements": {
        "container": {
          "image": "python",
          "tag": "3.8-slim-buster",
          "checksum": "sha256:53ad

### Publish the workflows:

- Two for the compute to the data assets
- One for the coordinator service

In [4]:
# Compute asset 1
metadata_workflow1 = utils.metadata_workflow_template()

metadata_workflow1["main"]["dateCreated"] = utils.date_now()
metadata_workflow1["main"]["workflow"]["stages"][0]["input"][0]["id"] = ASSET_COMPUTE_DID_1
metadata_workflow1["main"]["workflow"]["stages"][0]["transformation"]["id"] = ddo_algorithm.did

utils.print_json(metadata_workflow1)


ddo_workflow1 = nevermined.assets.create(metadata_workflow1, consumer_account, providers=[PROVIDER_ADDRESS])
print(f"Workflow DID: {ddo_workflow1.did}")

# Compute asset 2
metadata_workflow2 = utils.metadata_workflow_template()
metadata_workflow2["main"]["dateCreated"] = utils.date_now()
metadata_workflow2["main"]["workflow"]["stages"][0]["input"][0]["id"] = ASSET_COMPUTE_DID_2
metadata_workflow2["main"]["workflow"]["stages"][0]["transformation"]["id"] = ddo_algorithm.did

utils.print_json(metadata_workflow2)

ddo_workflow2 = nevermined.assets.create(metadata_workflow2, consumer_account, providers=[PROVIDER_ADDRESS])
print(f"Workflow DID: {ddo_workflow2.did}")

# Coordinator service
metadata_workflow_coordinator = utils.metadata_workflow_coordinator()
metadata_workflow_coordinator["main"]["dateCreated"] = utils.date_now()
metadata_workflow_coordinator["main"]["datePublished"] = utils.date_now()

utils.print_json(metadata_workflow_coordinator)


ddo_workflow_coordinator = nevermined.assets.create(metadata_workflow_coordinator, consumer_account, providers=[PROVIDER_ADDRESS])
print(f"Workflow DID: {ddo_workflow_coordinator.did}")

{
  "main": {
    "name": "Fraud Demo Workflow",
    "dateCreated": "2021-03-11T12:48:25Z",
    "author": "Nevermined Consumer",
    "license": "",
    "price": "0",
    "type": "workflow",
    "workflow": {
      "stages": [
        {
          "index": 0,
          "input": [
            {
              "index": 0,
              "id": "did:nv:ef129ded18f8bd94096156981726bab4e2f8afe7510617d33beb14e9cbafe17f"
            }
          ],
          "transformation": {
            "id": "did:nv:2483407d765fff264545fa5466f9745c546a3adb833218a9f5525abb7d36648c"
          }
        }
      ]
    }
  }
}
Workflow DID: did:nv:11139d39717e656cd2c901058273799bc74f81660fb38fb4a421bd1951795b13
{
  "main": {
    "name": "Fraud Demo Workflow",
    "dateCreated": "2021-03-11T12:48:28Z",
    "author": "Nevermined Consumer",
    "license": "",
    "price": "0",
    "type": "workflow",
    "workflow": {
      "stages": [
        {
          "index": 0,
          "input": [
            {
              "in

### Order computations:

- One for each data asset
- One for the coordinator service

In between we want for the blockchain to process the request.

In [5]:
keeper = Keeper.get_instance()

service_agreement_id1 = nevermined.assets.order(ASSET_COMPUTE_DID_1, ServiceTypesIndices.DEFAULT_COMPUTING_INDEX, consumer_account, consumer_account)
print(f"Service Agreement ID: {service_agreement_id1}")
utils.wait_for_event(keeper, service_agreement_id1)

service_agreement_id2 = nevermined.assets.order(ASSET_COMPUTE_DID_2, ServiceTypesIndices.DEFAULT_COMPUTING_INDEX, consumer_account, consumer_account)
print(f"Service Agreement ID: {service_agreement_id2}")
utils.wait_for_event(keeper, service_agreement_id2)

service_agreement_id_coordinator = nevermined.assets.order(ASSET_COORDINATOR_DID, ServiceTypesIndices.DEFAULT_COMPUTING_INDEX, consumer_account, consumer_account)
print(f"Service Agreement ID: {service_agreement_id_coordinator}")
utils.wait_for_event(keeper, service_agreement_id_coordinator)


Service Agreement ID: 0x9ebe02aa4925429c9288393569080a419c9fd205ab7c4e14b87157d15a26c2cd
Service Agreement ID: 0x559989b2c1c94b03b70e691fa410a1aba3d6242e68fd4d4e9e4a52f8e50fd788
Service Agreement ID: 0x201a04b95b7a4d9d889e93e723a53bc9d6b36f05cba24eeca2b6f11f7e7f22be


### Execute workflows

This orders the Nevermined to start the execution of the workflows:
- One for each data asset
- One for the coordinator service

In [6]:
execution_id1 = nevermined.assets.execute(
    service_agreement_id1,
    ASSET_COMPUTE_DID_1,
    ServiceTypesIndices.DEFAULT_COMPUTING_INDEX,
    consumer_account,
    ddo_workflow1.did
)
print(f"Execution ID: {execution_id1}")

execution_id2 = nevermined.assets.execute(
    service_agreement_id2,
    ASSET_COMPUTE_DID_2,
    ServiceTypesIndices.DEFAULT_COMPUTING_INDEX,
    consumer_account,
    ddo_workflow2.did
)
print(f"Execution ID: {execution_id2}")

execution_id_coordinator = nevermined.assets.execute(
    service_agreement_id_coordinator,
    ASSET_COORDINATOR_DID,
    ServiceTypesIndices.DEFAULT_COMPUTING_INDEX,
    consumer_account,
    ddo_workflow_coordinator.did
)
print(f"Execution ID: {execution_id_coordinator}")

Execution ID: nevermined-compute-bzs6m
Execution ID: nevermined-compute-685h9
Execution ID: nevermined-compute-56xkj


### Wait for compute jobs to finish

In [7]:
succeeded = set()
failed = set()
outputs = set()
while len(succeeded) < 3:
    for job_id in [execution_id1, execution_id2, execution_id_coordinator]:
        result = nevermined.assets.compute_status(service_agreement_id1, job_id, consumer_account)
        status = result["status"]
        print(f"{job_id}: {status}")

        if status == "Succeeded":
            succeeded.add(job_id)
            outputs.add(result["did"])
        elif status == "Failed":
            raise Exception("Some jobs failed")
    print()
    time.sleep(10)

for did in outputs:
    print(f"Output DID: {did}")

nevermined-compute-bzs6m: Running
nevermined-compute-685h9: Running
nevermined-compute-56xkj: Running

nevermined-compute-bzs6m: Running
nevermined-compute-685h9: Running
nevermined-compute-56xkj: Running

nevermined-compute-bzs6m: Running
nevermined-compute-685h9: Running
nevermined-compute-56xkj: Running

nevermined-compute-bzs6m: Running
nevermined-compute-685h9: Running
nevermined-compute-56xkj: Running

nevermined-compute-bzs6m: Running
nevermined-compute-685h9: Running
nevermined-compute-56xkj: Running

nevermined-compute-bzs6m: Running
nevermined-compute-685h9: Running
nevermined-compute-56xkj: Running

nevermined-compute-bzs6m: Running
nevermined-compute-685h9: Running
nevermined-compute-56xkj: Running

nevermined-compute-bzs6m: Running
nevermined-compute-685h9: Running
nevermined-compute-56xkj: Running

nevermined-compute-bzs6m: Running
nevermined-compute-685h9: Running
nevermined-compute-56xkj: Running

nevermined-compute-bzs6m: Running
nevermined-compute-685h9: Running
never

### Download the results

In [8]:
for did in outputs:
    print(f"Downloading: {did}")
    nevermined.assets.download(did, ServiceTypesIndices.DEFAULT_ACCESS_INDEX, consumer_account, "./")

Downloading: did:nv:abaf4bb3467248b65c8cf3bc703f1978061256445210f4720d4543c7a155dfda
Downloading: did:nv:ac2864fb7f68b079c21ee9b1904c319398289ad6c671dd989597fe48060f5d92
Downloading: did:nv:350410e9426e01029ed3e9a2b06b8c525f734e91d83242fa02169c808ed1141b


In [9]:
nevermined.assets.compute_status(service_agreement_id1, job_id, consumer_account)

{'did': 'did:nv:350410e9426e01029ed3e9a2b06b8c525f734e91d83242fa02169c808ed1141b',
 'finishedAt': '2021-03-11T12:55:43+00:00Z',
 'pods': [{'finishedAt': '2021-03-11T12:54:11+00:00Z',
   'podName': 'aggregator',
   'startedAt': '2021-03-11T12:49:37+00:00Z',
   'status': 'Succeeded'},
  {'finishedAt': '2021-03-11T12:54:16+00:00Z',
   'podName': 'coordinator',
   'startedAt': '2021-03-11T12:49:37+00:00Z',
   'status': 'Succeeded'},
  {'finishedAt': '2021-03-11T12:49:32+00:00Z',
   'podName': 'configurator',
   'startedAt': '2021-03-11T12:49:27+00:00Z',
   'status': 'Succeeded'},
  {'finishedAt': '2021-03-11T12:55:33+00:00Z',
   'podName': 'publishing',
   'startedAt': '2021-03-11T12:54:21+00:00Z',
   'status': 'Succeeded'}],
 'startedAt': '2021-03-11T12:49:27+00:00Z',
 'status': 'Succeeded'}